In [ ]:
#!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np

stock_data = pd.read_csv("../data/collated.csv")

In [ ]:
stock_data.head()

,Unnamed: 0,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector
0,0,7/9/2022,-0.38,-0.35,0.0741,1/6/2022,-0.122655,0.106351,-0.269231,1,GME,Consumer Cyclical
1,1,1/6/2022,-0.36,-0.52,-0.4325,17/3/2022,0.579668,0.219666,-0.217391,0,GME,Consumer Cyclical
2,2,17/3/2022,0.21,-0.46,-3.2249,8/12/2021,-0.535234,0.180643,-1.600000,0,GME,Consumer Cyclical
3,3,8/12/2021,-0.13,-0.35,-1.6977,8/9/2021,-0.160201,0.086445,-0.315789,0,GME,Consumer Cyclical
4,4,8/9/2021,-0.17,-0.19,-0.1446,9/6/2021,-0.324167,0.164528,0.545455,0,GME,Consumer Cyclical


In [ ]:
import datetime
print(len(stock_data))
date_list = []
for i in range(len(stock_data)):
    new_date_format = datetime.datetime.strptime(stock_data.iloc[i]["Earnings Date"], '%d/%m/%Y').strftime('%Y-%m-%d')
    date_list.append(new_date_format)
stock_data["Earnings Date Different format"] = pd.Series(date_list)
stock_data.head()

979


,Unnamed: 0,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector,Earnings Date Different format
0,0,7/9/2022,-0.38,-0.35,0.0741,1/6/2022,-0.122655,0.106351,-0.269231,1,GME,Consumer Cyclical,2022-09-07
1,1,1/6/2022,-0.36,-0.52,-0.4325,17/3/2022,0.579668,0.219666,-0.217391,0,GME,Consumer Cyclical,2022-06-01
2,2,17/3/2022,0.21,-0.46,-3.2249,8/12/2021,-0.535234,0.180643,-1.600000,0,GME,Consumer Cyclical,2022-03-17
3,3,8/12/2021,-0.13,-0.35,-1.6977,8/9/2021,-0.160201,0.086445,-0.315789,0,GME,Consumer Cyclical,2021-12-08
4,4,8/9/2021,-0.17,-0.19,-0.1446,9/6/2021,-0.324167,0.164528,0.545455,0,GME,Consumer Cyclical,2021-09-08


In [ ]:
class SupressSettingWithCopyWarning:
    def __enter__(self):
        pd.options.mode.chained_assignment = None

    def __exit__(self, *args):
        pd.options.mode.chained_assignment = 'warn'

In [ ]:
def stock_prices(ticker, curr_date):
    MAX_NUM_TRY = 100
    for _ in range(MAX_NUM_TRY):
        curr_ticker = yf.Ticker(ticker)
        if len(curr_ticker.history()) > 0:
            break
    start_date = datetime.datetime.strptime(curr_date, '%Y-%m-%d').date() - datetime.timedelta(days=6)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = datetime.datetime.strptime(curr_date, '%Y-%m-%d').date() + datetime.timedelta(days=6)
    end_date = end_date.strftime('%Y-%m-%d')
    stock_prices_dates = curr_ticker.history(start=start_date, end=end_date)
    display(stock_prices_dates)
    for i in range(len(stock_prices_dates)):
        if stock_prices_dates.index[i].date() == datetime.datetime.strptime(curr_date, '%Y-%m-%d').date():
            two_day_before = stock_prices_dates.iloc[i-2]["Close"] #2 days before EPS release (before market close)
            one_day_before = stock_prices_dates.iloc[i-1]["Close"] #1 day before EPS release (before market close)
            one_day_after = stock_prices_dates.iloc[i]["Close"] #the day of the EPS release (before market close)
            two_day_after = stock_prices_dates.iloc[i+1]["Close"] #1 day after EPS release (before market close)
    return two_day_before, one_day_before, one_day_after, two_day_after

stock_prices("GME", "2022-09-07") #check function is working

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-09-01 00:00:00-04:00,28.000000,28.910000,26.950001,27.629999,5293900,0,0
2022-09-02 00:00:00-04:00,28.260000,28.740000,26.930000,27.360001,4675200,0,0
2022-09-06 00:00:00-04:00,25.750000,26.719999,24.670000,25.139999,7010200,0,0
2022-09-07 00:00:00-04:00,24.730000,25.190001,23.420000,24.040001,12504700,0,0
2022-09-08 00:00:00-04:00,25.000000,26.740000,24.070000,25.830000,14142200,0,0
2022-09-09 00:00:00-04:00,26.299999,29.080000,26.100000,28.920000,7899800,0,0
2022-09-12 00:00:00-04:00,29.030001,30.340000,28.000000,29.240000,6175200,0,0


(27.360000610351562,
 25.139999389648438,
 24.040000915527344,
 25.829999923706055)

In [ ]:
two_day_before_list = []
one_day_before_list = []
one_day_after_list = []
two_day_after_list = []

In [ ]:
counter = 0 #track progress in case of error from yfinance
for i in range(0, len(stock_data)):
    print(counter)
    ticker = stock_data.iloc[i]["Ticker"]
    curr_date = stock_data.iloc[i]["Earnings Date Different format"]
    print(curr_date)
    two_day_before, one_day_before, one_day_after, two_day_after = stock_prices(ticker, curr_date)
    two_day_before_list.append(two_day_before)
    one_day_before_list.append(one_day_before)
    one_day_after_list.append(one_day_after)
    two_day_after_list.append(two_day_after)
    counter += 1
    import time 
    time.sleep(2) #40

825
2022-03-21


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-03-15 00:00:00-04:00,25.520000,29.750000,23.209999,27.309999,56774300,0,0
2022-03-16 00:00:00-04:00,37.119999,42.970001,35.639999,42.619999,79707700,0,0
2022-03-17 00:00:00-04:00,40.849998,41.070000,36.810001,39.369999,32409300,0,0
2022-03-18 00:00:00-04:00,39.660000,45.970001,39.160000,42.599998,43263700,0,0
2022-03-21 00:00:00-04:00,44.939999,45.980000,38.500000,39.990002,52251800,0,0
2022-03-22 00:00:00-04:00,42.779999,49.430000,41.830002,47.529999,55714600,0,0
2022-03-23 00:00:00-04:00,46.830002,55.380001,46.240002,49.400002,52480900,0,0
2022-03-24 00:00:00-04:00,47.709999,48.310001,44.520000,44.939999,43293900,0,0
2022-03-25 00:00:00-04:00,42.590000,44.790001,41.939999,44.240002,25979400,0,0


826
2021-11-26


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-11-22 00:00:00-05:00,81.930000,84.059998,78.019997,79.519997,9646800,0,0
2021-11-23 00:00:00-05:00,78.949997,81.790001,78.250000,79.269997,5965900,0,0
2021-11-24 00:00:00-05:00,79.900002,82.099998,79.059998,81.360001,6420100,0,0
2021-11-26 00:00:00-05:00,73.910004,74.839996,65.510002,68.459999,23275800,0,0
2021-11-29 00:00:00-05:00,68.169998,72.379997,67.449997,71.959999,13258200,0,0
2021-11-30 00:00:00-05:00,70.019997,70.739998,65.900002,66.500000,11687100,0,0
2021-12-01 00:00:00-05:00,68.419998,68.839996,61.860001,62.110001,12936000,0,0


827
2021-08-24


...

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-07 00:00:00-04:00,124.163430,126.369395,124.069553,125.383759,4424500,0,0
2020-07-08 00:00:00-04:00,125.937576,126.435092,124.998864,125.993896,3798200,0,0
2020-07-09 00:00:00-04:00,124.754820,125.731089,123.656529,124.576477,4005200,0,0
2020-07-10 00:00:00-04:00,124.257274,126.622832,123.234082,126.219193,5932800,0,0
2020-07-13 00:00:00-04:00,128.387614,129.607942,126.228574,126.641609,8960000,0,0
2020-07-14 00:00:00-04:00,126.341239,128.556600,126.022080,127.214249,6766800,0,0
2020-07-15 00:00:00-04:00,127.054664,127.843179,125.083363,125.496399,5561300,0,0
2020-07-16 00:00:00-04:00,125.843728,126.125344,124.895634,125.674767,3788100,0,0
2020-07-17 00:00:00-04:00,126.397556,126.782431,125.364981,126.406952,3229300,0,0


977
2020-04-28


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-22 00:00:00-04:00,123.365444,124.511163,122.229035,123.533104,4084600,0,0
2020-04-23 00:00:00-04:00,123.421340,124.948970,121.120579,121.334816,5527700,0,0
2020-04-24 00:00:00-04:00,121.903023,125.582388,121.698096,125.153900,4063100,0,0
2020-04-27 00:00:00-04:00,125.209795,125.945674,124.716111,125.247063,4009200,0,0
2020-04-28 00:00:00-04:00,127.408088,128.246416,125.433351,126.979614,6674200,0,0
2020-04-29 00:00:00-04:00,127.398768,127.594372,123.793924,124.222412,6706800,0,0
2020-04-30 00:00:00-04:00,123.337511,124.045432,121.483857,123.225723,6783000,0,0
2020-05-01 00:00:00-04:00,122.135878,123.169823,120.645516,121.223030,4488000,0,0


978
2020-02-13


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-02-07 00:00:00-05:00,133.474317,134.825165,133.076473,134.501328,3138500,0,0
2020-02-10 00:00:00-05:00,135.019426,135.472796,134.279250,134.769623,3775600,0,0
2020-02-11 00:00:00-05:00,135.278534,135.417313,134.038709,135.158249,3870500,0,0
2020-02-12 00:00:00-05:00,135.713381,135.768907,134.482832,135.158249,4217400,0,0
2020-02-13 00:00:00-05:00,134.390286,136.129732,132.789635,135.519073,5041400,0,0
2020-02-14 00:00:00-05:00,135.611617,136.148240,135.028715,136.000214,3555600,0,0
2020-02-18 00:00:00-05:00,136.111241,136.194508,134.529084,134.714142,4938700,0,0


In [ ]:
len(two_day_before_list)

979

In [ ]:
stock_data["two_day_before"] = pd.Series(two_day_before_list)
stock_data["one_day_before"] = pd.Series(one_day_before_list)
stock_data["one_day_after"] = pd.Series(one_day_after_list)
stock_data["two_day_after"] = pd.Series(two_day_after_list)

In [ ]:
stock_data.head()

,Unnamed: 0,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,EPS Beats,Ticker,Sector,Earnings Date Different format,two_day_before,one_day_before,one_day_after,two_day_after
0,0,7/9/2022,-0.38,-0.35,0.0741,1/6/2022,-0.122655,0.106351,-0.269231,1,GME,Consumer Cyclical,2022-09-07,27.360001,25.139999,24.040001,25.830000
1,1,1/6/2022,-0.36,-0.52,-0.4325,17/3/2022,0.579668,0.219666,-0.217391,0,GME,Consumer Cyclical,2022-06-01,34.302502,31.184999,30.350000,33.500000
2,2,17/3/2022,0.21,-0.46,-3.2249,8/12/2021,-0.535234,0.180643,-1.600000,0,GME,Consumer Cyclical,2022-03-17,20.660000,21.715000,21.924999,22.697500
3,3,8/12/2021,-0.13,-0.35,-1.6977,8/9/2021,-0.160201,0.086445,-0.315789,0,GME,Consumer Cyclical,2021-12-08,41.779999,44.452499,43.412498,38.939999
4,4,8/9/2021,-0.17,-0.19,-0.1446,9/6/2021,-0.324167,0.164528,0.545455,0,GME,Consumer Cyclical,2021-09-08,50.687500,49.750000,49.700001,49.794998


In [ ]:
stock_data["first_strat"] = (stock_data["one_day_after"] - stock_data["two_day_before"]) / stock_data["two_day_before"]
stock_data["second_strat"] = (stock_data["two_day_after"] - stock_data["two_day_before"]) / stock_data["two_day_before"]
stock_data["third_strat"] = (stock_data["one_day_after"] - stock_data["one_day_before"]) / stock_data["one_day_before"]
stock_data["fourth_strat"] = (stock_data["two_day_after"] - stock_data["one_day_before"]) / stock_data["one_day_before"]

In [ ]:
stock_data["first_strat_profit"] = np.where(stock_data["EPS Beats"] == 1, stock_data["first_strat"], -stock_data["first_strat"])
stock_data["second_strat_profit"] = np.where(stock_data["EPS Beats"] == 1, stock_data["second_strat"], -stock_data["second_strat"])
stock_data["third_strat_profit"] = np.where(stock_data["EPS Beats"] == 1, stock_data["third_strat"], -stock_data["third_strat"])
stock_data["fourth_strat_profit"] = np.where(stock_data["EPS Beats"] == 1, stock_data["fourth_strat"], -stock_data["fourth_strat"])

In [ ]:
stock_data.to_csv("updated_collated.csv", encoding='utf-8', header=True) #store to csv file

In [ ]:
stock_data = pd.read_csv("../data/updated_collated.csv")

In [ ]:
stock_data['year'] = stock_data["Earnings Date"].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').year)

In [ ]:
stock_data

,Unnamed: 0.1,Unnamed: 0,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Previous Earnings Date,Price Change,Coefficient Variation of Stock Price,Percentage Change from prev reported EPS,...,one_day_after,two_day_after,first_strat,second_strat,third_strat,fourth_strat,first_strat_profit,second_strat_profit,third_strat_profit,fourth_strat_profit
0,0,0,7/9/2022,-0.38,-0.35,0.0741,1/6/2022,-0.122655,0.106351,-0.269231,...,24.040001,25.830000,-0.121345,-0.055921,-0.043755,0.027446,-0.121345,-0.055921,-0.043755,0.027446
1,1,1,1/6/2022,-0.36,-0.52,-0.4325,17/3/2022,0.579668,0.219666,-0.217391,...,30.350000,33.500000,-0.115225,-0.023395,-0.026776,0.074234,0.115225,0.023395,0.026776,-0.074234
2,2,2,17/3/2022,0.21,-0.46,-3.2249,8/12/2021,-0.535234,0.180643,-1.600000,...,21.924999,22.697500,0.061229,0.098621,0.009671,0.045245,-0.061229,-0.098621,-0.009671,-0.045245
3,3,3,8/12/2021,-0.13,-0.35,-1.6977,8/9/2021,-0.160201,0.086445,-0.315789,...,43.412498,38.939999,0.039074,-0.067975,-0.023396,-0.124009,-0.039074,0.067975,0.023396,0.124009
4,4,4,8/9/2021,-0.17,-0.19,-0.1446,9/6/2021,-0.324167,0.164528,0.545455,...,49.700001,49.794998,-0.019482,-0.017608,-0.001005,0.000904,0.019482,0.017608,0.001005,-0.000904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,974,7,11/2/2021,1.46,1.47,0.0096,1/10/2020,0.014390,0.025595,-0.120482,...,128.518112,127.470680,-0.033166,-0.041046,-0.019826,-0.027814,-0.033166,-0.041046,-0.019826,-0.027814
975,975,8,1/10/2020,1.49,1.66,0.1156,13/7/2020,0.023988,0.016441,0.128788,...,133.121048,130.530457,0.026538,0.006562,0.015873,-0.003896,0.026538,0.006562,0.015873,-0.003896
976,976,9,13/7/2020,1.25,1.32,0.0543,28/4/2020,-0.005354,0.012858,0.168224,...,126.641609,127.214249,0.016577,0.021174,0.003347,0.007884,0.016577,0.021174,0.003347,0.007884
977,977,10,28/4/2020,1.03,1.07,0.0398,13/2/2020,-0.074019,0.081405,-0.289655,...,126.979614,124.222412,0.014588,-0.007443,0.013833,-0.008181,0.014588,-0.007443,0.013833,-0.008181


In [ ]:
#remove_2022 = stock_data.drop(stock_data[stock_data['year'] == 2022].index, inplace=True) 
#this is just to check performance without this year's market as it has been on downtrend. (not much difference.)

In [ ]:
#The following are the decriptive statistics of the returns if we open a buy position and close after the specified days, 
#assuming the stock beats EPS and if we open a short position and close after the specified days assuming the stock misses EPS
stock_data["first_strat_profit"].describe()

count    979.000000
mean       0.013191
std        0.079589
min       -0.515041
25%       -0.022959
50%        0.008758
75%        0.044497
max        0.568585
Name: first_strat_profit, dtype: float64

In [ ]:
stock_data["second_strat_profit"].describe()

count    979.000000
mean       0.023666
std        0.129636
min       -0.622529
25%       -0.041052
50%        0.018944
75%        0.082129
max        0.553176
Name: second_strat_profit, dtype: float64

In [ ]:
stock_data["third_strat_profit"].describe()

count    979.000000
mean       0.011116
std        0.062089
min       -0.259786
25%       -0.019514
50%        0.008009
75%        0.036477
max        0.294675
Name: third_strat_profit, dtype: float64

In [ ]:
stock_data["fourth_strat_profit"].describe()

count    979.000000
mean       0.021247
std        0.118802
min       -0.599356
25%       -0.037627
50%        0.014576
75%        0.078114
max        0.620642
Name: fourth_strat_profit, dtype: float64

In [ ]:
print("The second_strat_profit is the most promising, with an average return of 2.37% per trade")
print("The second investment strategy is:")
print("- Assuming EPS Beats: buying the stock 2 days before the EPS release and selling at the end of day of the EPS release")
print("- Assuming EPS Misses: short selling the stock 2 days before the EPS release and closing at the end of the day of the EPS release")

#To show that this investment strategy has some usefulness, we compare to the baseline where we start a trade assuming every stock beats
#The following are the decriptive statistics of the returns if we open a buy position and close after the specified days, 
#regardless of if the stock beats or not.
print()
print("In comparison, we look at the baseline of always assuming EPS beats")
stock_data["first_strat"].describe()

The second_strat_profit is the most promising, with an average return of 2.37% per trade
The second investment strategy is:
- Assuming EPS Beats: buying the stock 2 days before the EPS release and selling at the end of day of the EPS release
- Assuming EPS Misses: short selling the stock 2 days before the EPS release and closing at the end of the day of the EPS release

In comparison, we look at the baseline of always assuming EPS beats


count    979.000000
mean       0.006194
std        0.080438
min       -0.298029
25%       -0.029019
50%        0.004655
75%        0.039678
max        0.568585
Name: first_strat, dtype: float64

In [ ]:
stock_data["second_strat"].describe()

count    979.000000
mean       0.004978
std        0.131687
min       -0.622529
25%       -0.056887
50%       -0.000422
75%        0.065348
max        0.576016
Name: second_strat, dtype: float64

In [ ]:
stock_data["third_strat"].describe()

count    979.000000
mean       0.001103
std        0.063068
min       -0.274471
25%       -0.025937
50%        0.003347
75%        0.028881
max        0.294675
Name: third_strat, dtype: float64

In [ ]:
stock_data["fourth_strat"].describe()

count    979.000000
mean      -0.000251
std        0.120688
min       -0.599356
25%       -0.058574
50%       -0.000477
75%        0.061342
max        0.620642
Name: fourth_strat, dtype: float64

In [ ]:
print("As can be seen, all of the above four simple investment strategies have an average return of near zero percent")
print("in comparison, by following the simple investment strategies when the stock beats EPS,")
print("and inversing the strategies when the stock misses EPS, there is an average return of two percent")
print("\nAs the second investment strategy has the best performance, we will stick with this our backtesting")

As can be seen, all of the above four simple investment strategies have an average return of near zero percent
in comparison, by following the simple investment strategies when the stock beats EPS,
and inversing the strategies when the stock misses EPS, there is an average return of two percent

As the second investment strategy has the best performance, we will stick with this our backtesting
